# setup

In [1]:
import os

if os.getcwd().split('/')[-1] == 'notebooks':
    os.chdir('..')

In [2]:
import torch
import torch.nn as nn

from models.baseline import ResnetBaseline

In [3]:
# model_label = 'code15'
# epochs = 15

# from dataloaders.code15 import CODE as DS
# from dataloaders.code15 import CODEsplit as DSsplit
# from runners.multiclass import Runner
# model = ResnetBaseline(n_classes = 6)

In [4]:
# model_label = 'code15normal'
# epochs = 15

# from dataloaders.code15 import CODE as DS
# from dataloaders.code15 import CODEsplit as DSsplit
# from runners.normal import Runner
# model = ResnetBaseline(n_classes = 1)

In [5]:
# model_label = 'code15flag1davb'
# epochs = 10
# from runners.multiclass import Runner
# from dataloaders.code15 import CODE as DS
# from dataloaders.code15 import CODEsplit as DSsplit
# model = ResnetBaseline(n_classes = 6)

In [6]:
# model_label = 'code15flag1davb'
# epochs = 10
# from runners.multiclass import Runner
# from dataloaders.code15 import CODE as DS
# from dataloaders.code15 import CODEsplit as DSsplit
# model = ResnetBaseline(n_classes = 6)

In [7]:
model_label = 'code15flagnormal'
epochs = 1
from runners.normal import Runner
from dataloaders.code15 import CODE as DS
from dataloaders.code15 import CODEsplit as DSsplit
model = ResnetBaseline(n_classes = 1)

# init

In [8]:
# database = DS(metadata_path = '/home/josegfer/datasets/code/output/metadata_1davb.csv')
database = DS(metadata_path = '/home/josegfer/datasets/code/output/metadata_normal.csv')
model = torch.load('output/{}/{}.pt'.format(model_label, model_label))

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [10]:
runner = Runner(device = device, model = model, database = database, split = DSsplit, model_label = model_label)

# run

In [63]:
runner.train(epochs)

  0%|          | 0/2512 [00:00<?, ?it/s]

-- epoch 0


100%|██████████| 97/97 [01:16<00:00,  1.27it/s]


[[0.4985295777582818, 0.47676371360562514]]
new checkpoint with val loss: 0.47676371360562514
exporting partial model at epoch 0


In [11]:
runner.eval()

100%|██████████| 94/94 [01:14<00:00,  1.27it/s]


exporting last model


# test

In [12]:
import pandas as pd
import h5py
import numpy as np

from torch.utils.data import Dataset

class CODEtest(Dataset):
    def __init__(self, hdf5_path = '/home/josegfer/datasets/code/data/codetest/data/ecg_tracings.hdf5', 
                 metadata_path = '/home/josegfer/datasets/code/data/codetest/data/annotations/gold_standard.csv',
                 tracing_col = 'tracings', output_col = ['1dAVb', 'RBBB', 'LBBB', 'SB', 'AF', 'ST']):
        self.hdf5_file = h5py.File(hdf5_path, 'r')
        self.metadata = pd.read_csv(metadata_path)

        self.tracing_col = tracing_col
        self.output_col = output_col
    
    def __len__(self):
        return len(self.metadata)
    
    def __getitem__(self, idx):
        return {'x': self.hdf5_file[self.tracing_col][idx],
                'y': ~np.sum(self.metadata[self.output_col].loc[idx].values, dtype = bool).reshape(-1),}

In [13]:
from utils import json_dump
from hparams import BATCH_SIZE, NUM_WORKERS

In [14]:
tst_ds = CODEtest()
val_dl = torch.utils.data.DataLoader(runner.val_ds, batch_size = BATCH_SIZE, 
                                     shuffle = False, num_workers = NUM_WORKERS)
tst_dl = torch.utils.data.DataLoader(tst_ds, batch_size = BATCH_SIZE, 
                                     shuffle = False, num_workers = NUM_WORKERS)

In [15]:
best_f1s, best_thresholds = runner._synthesis(val_dl, best_thresholds = None)
all_binary_results, all_true_labels, metrics_dict = runner._synthesis(tst_dl, best_thresholds)
json_dump(metrics_dict, model_label, test = True)

100%|██████████| 7/7 [00:02<00:00,  2.34it/s]
